In [6]:
try:
    # Load in openpolicedata library and rename it as opd for convenience
    import openpolicedata as opd
except:
    # Import failed
    # Typically, openpolicedata is already installed
    # On Jupyter Lite, we have to install it
    %pip install openpolicedata
    import openpolicedata as opd

# This block of code won't be needed when the IRE talk happens
try:
    import ssl
except:
    %pip install ssl
    import ssl

# This is only necessary for running in Jupyter Lite which cannot access the internet
# It requires the OPD data source CSV file to be located in the same directory
# Download the the source table from: https://github.com/openpolicedata/opd-data/blob/main/opd_source_table.csv
try:
    # Try to access OPD datasets
    opd.datasets.query()
except:
    # If not available, load failed. Load datasets from local file
    opd.datasets.datasets = opd.datasets._build("opd_source_table.csv")

In [14]:
# Get all the datasets available in OPD
datasets = opd.datasets.query()

print(f"There are {opd.datasets.num_unique()} datasets available in OPD")
print("Details of the first 5 rows of the datasets table are:")
datasets.head(n=2)

There are 365 datasets available in OPD
Details of the first 5 rows of the datasets table are:


,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Description,source_url,readme,URL,Year,DataType,date_field,dataset_id,agency_field,min_version
0,Arizona,Gilbert,Gilbert,Gilbert Police Department,CALLS FOR SERVICE,2006-11-15,2023-05-14,05/15/2023,<NA>,https://data.gilbertaz.gov/maps/2dcb4c20c9a444...,<NA>,https://maps.gilbertaz.gov/arcgis/rest/service...,MULTIPLE,ArcGIS,EventDate,<NA>,<NA>,<NA>
1,Arizona,Gilbert,Gilbert,Gilbert Police Department,EMPLOYEE,NaT,NaT,05/15/2023,A data set of all employees that have previous...,https://data.gilbertaz.gov/datasets/TOG::gilbe...,<NA>,https://services1.arcgis.com/JLuzSHjNrLL4Okwb/...,NONE,ArcGIS,<NA>,<NA>,<NA>,<NA>


In [16]:
print("The types of data available in OPD are:")
unique_tables = datasets["TableType"].unique()
unique_tables

The types of data available in OPD are:


array(['CALLS FOR SERVICE', 'EMPLOYEE', 'STOPS', 'INCIDENTS', 'ARRESTS',
       'OFFICER-INVOLVED SHOOTINGS', 'TRAFFIC CITATIONS', 'USE OF FORCE',
       'OFFICER-INVOLVED SHOOTINGS - CIVILIANS',
       'OFFICER-INVOLVED SHOOTINGS - INCIDENTS',
       'OFFICER-INVOLVED SHOOTINGS - OFFICERS', 'TRAFFIC STOPS',
       'USE OF FORCE - INCIDENTS', 'USE OF FORCE - CIVILIANS/OFFICERS',
       'CRASHES', 'CITATIONS', 'COMPLAINTS', 'VEHICLE PURSUITS',
       'CRASHES - CIVILIANS', 'CRASHES - INCIDENTS', 'PEDESTRIAN STOPS',
       'FIELD CONTACTS', 'TRAFFIC WARNINGS', 'CRASHES - NONMOTORIST',
       'LAWSUITS', 'TRAFFIC ARRESTS', 'COMPLAINTS - ALLEGATIONS',
       'COMPLAINTS - BACKGROUND', 'COMPLAINTS - OFFICERS',
       'COMPLAINTS - PENALTIES', 'CRASHES - VEHICLES',
       'USE OF FORCE - CIVILIANS', 'USE OF FORCE - OFFICERS',
       'COMPLAINTS - CIVILIANS', 'PEDESTRIAN CITATIONS',
       'PEDESTRIAN ARRESTS', 'PEDESTRIAN WARNINGS'], dtype=object)

In [18]:
stops_datasets_va = opd.datasets.query(state="Virginia", table_type="STOPS")
stops_datasets_va

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Description,source_url,readme,URL,Year,DataType,date_field,dataset_id,agency_field,min_version
834,Virginia,Dumfries,Dumfries,Dumfries Police Department,STOPS,2021-07-01,2023-03-31,05/15/2023,<NA>,https://data.dumfriesva.gov/Government/Dumfrie...,<NA>,data.dumfriesva.gov,MULTIPLE,Socrata,date,usdq-at3n,<NA>,<NA>
851,Virginia,Virginia,MULTIPLE,NaN,STOPS,2020-07-01,2023-03-31,05/15/2023,A data collection consisting of all traffic an...,https://data.virginia.gov/Public-Safety/Commun...,<NA>,data.virginia.gov,MULTIPLE,Socrata,incident_date,2c96-texw,agency_name,<NA>


In [23]:
year = 2022
agency = "Richmond Police Department"

src = opd.Source("Virginia")
try:
    t = src.load_from_url(year, table_type="STOPS", agency=agency)
except:
    # Jupyter Lite's internet connection often times out.
    # This is not an issue when not using Jupyter Lite
    # Load from local file instead
    t = src.load_from_csv(year, table_type="STOPS", agency=agency)

print(f"There were {len(t.table)} stops by {agency} in {year}")
print("Here are the first 5 rows from the table:")
t.table.head()

There were 1983 stops by Richmond Police Department in 2022
Here are the first 5 rows from the table:


,incident_date,agency_name,jurisdiction,reason_for_stop,person_type,race,ethnicity,age,gender,english_speaking,action_taken,specific_violation,person_searched,vehicle_searched,physical_force_by_officer,physical_force_by_subject,residency
0,2022-01-01,Richmond Police Department,RICHMOND CITY,TRAFFIC VIOLATION,DRIVER,WHITE,NOT HISPANIC OR LATINO,39,MALE,YES,CITATION/SUMMONS,46.2-646,NO,NO,NO,NO,NaN
1,2022-01-01,Richmond Police Department,RICHMOND CITY,EQUIPMENT VIOLATION,DRIVER,WHITE,NOT HISPANIC OR LATINO,38,MALE,YES,CITATION/SUMMONS,46.2-1158,NO,NO,NO,NO,NaN
2,2022-01-01,Richmond Police Department,RICHMOND CITY,TRAFFIC VIOLATION,DRIVER,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,20,FEMALE,YES,WARNING ISSUED,46.2-875,NO,NO,NO,NO,NaN
3,2022-01-01,Richmond Police Department,RICHMOND CITY,TRAFFIC VIOLATION,DRIVER,WHITE,NOT HISPANIC OR LATINO,21,FEMALE,YES,CITATION/SUMMONS,46.2-870,NO,NO,NO,NO,NaN
4,2022-01-01,Richmond Police Department,RICHMOND CITY,TRAFFIC VIOLATION,DRIVER,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,43,MALE,YES,CITATION/SUMMONS,46.2-862(i),NO,NO,NO,NO,NaN
